In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
cur_normal = pd.read_csv('2.2kW_cur_L-DSF-01_training_with_label.csv')
vib_normal = pd.read_csv('2.2kW_vib_L-DSF-01_training_with_label.csv')
cur_validation = pd.read_csv('2.2kW_cur_L-DSF-01_validation_with_label.csv')
vib_validation = pd.read_csv('2.2kW_vib_L-DSF-01_validation_with_label.csv')

In [4]:
print(len(cur_normal))
print(len(vib_normal))
print(len(cur_validation))
print(len(vib_validation))

36768
24487
4599
3064


In [6]:
np_cur_normal = cur_normal.to_numpy()
np_vib_normal = vib_normal.to_numpy()
np_cur_validation = cur_validation.to_numpy()
np_vib_validation = vib_validation.to_numpy()

# Extract Labels

In [44]:
normal_cur_labels = np_cur_normal[:,-1]
normal_vib_labels = np_vib_normal[:,-1]
val_cur_labels = np_cur_validation[:, -1]
val_vib_labels = np_vib_validation[:, -1]

print(normal_cur_labels)
print(normal_vib_labels)
print(val_cur_labels)
print(val_vib_labels)

[0 0 0 ... 3 3 3]
[0 0 0 ... 3 3 3]
[0 0 0 ... 3 3 3]
[0 0 0 ... 3 3 3]


In [45]:
cur_normal_dropped = cur_normal.drop(columns = ['Date','WATT','LABEL'])
vib_normal_dropped = vib_normal.drop(columns = ['Date','WATT','LABEL'])
cur_validation_dropped = cur_validation.drop(columns = ['Date','WATT','LABEL'])
vib_validation_dropped = vib_validation.drop(columns = ['Date','WATT','LABEL'])

In [47]:
cur_normal_dropped.head(3)

,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,R_RMS,...,T_1x,R_2x,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x
0,3.403320,3.123047,4.043945,2.057587,1.889543,2.245951,6.726562,6.166016,7.607422,2.296682,...,0.014614,3.282771,3.058104,3.579351,0.022117,0.019693,0.023455,0.031669,0.027953,0.024256
1,3.283203,3.083008,4.003906,2.059709,1.883498,2.254960,6.526367,6.005859,7.487305,2.287841,...,0.015374,3.433224,3.183656,3.761500,0.013864,0.011091,0.014099,0.030992,0.039196,0.035626
2,3.283203,3.083008,3.963867,2.059209,1.884539,2.255280,6.526367,6.085938,7.407227,2.288444,...,0.019783,3.432319,3.184894,3.755824,0.022333,0.019714,0.020837,0.024400,0.026358,0.023356


In [48]:
cur_normal_dropped_np = cur_normal_dropped.to_numpy()
vib_normal_dropped_np = vib_normal_dropped.to_numpy()
cur_validation_dropped_np = cur_validation_dropped.to_numpy()
vib_validation_dropped_np = vib_validation_dropped.to_numpy()

# 1.  Train에서 Test set 분리 : 1. CURRENT

In [50]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cur_normal_dropped_np, normal_cur_labels, stratify=normal_cur_labels, test_size=0.2, random_state=42)

In [51]:
print(x_train[0])
print(x_test[0])
print(y_train[0])
print(y_test[0])

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

[ 3.24316406e+00  3.16308594e+00  3.96386719e+00  2.01802881e+00
  1.86465918e+00  2.20753369e+00  6.48632812e+00  6.20605469e+00
  7.36718750e+00  2.24058769e+00  2.08956767e+00  2.46386464e+00
  6.39920396e-03 -7.89028211e-04 -7.12709533e-03  1.47027723e+00
  1.50019839e+00  1.50548275e+00  2.89492268e+00  2.97001853e+00
  2.99009426e+00  1.11028528e+00  1.12061641e+00  1.11611644e+00
  1.60709503e+00  1.69633463e+00  1.79560892e+00 -3.14159265e+00
 -3.14159265e+00  3.14159265e+00  1.60696777e-01  2.10028418e+00
 -2.26098096e+00  1.13101505e-02  1.01238237e-02  1.31138674e-02
  3.21958821e+00  3.00243212e+00  3.52495620e+00  1.22113268e-02
  1.18758799e-02  1.16075196e-02  3.26346269e-02  3.53758430e-02
  3.65411011e-02]
[ 3.36328125e+00  3.20312500e+00  4.16406250e+00  2.13224023e+00
  1.93662939e+00  2.34612891e+00  6.72656250e+00  6.28613281e+00
  7.84765625e+00  2.39082191e+00  2.18733054e+00  2.64398306e+00
 -5.31305087e-03 -6.09845670e-03  3.81981107e-03  1.48597871e+00
  1.511

# 2. normalize

In [52]:
from sklearn.preprocessing import StandardScaler

# 변형 객체 생성
std_scaler = StandardScaler()

# 훈련데이터의 모수 분포 저장
std_scaler.fit(x_train)

# 훈련 데이터 스케일링
X_train_scaled = std_scaler.transform(x_train)

# 테스트 데이터의 스케일링
X_test_scaled = std_scaler.transform(x_test)

# 스케일링 된 결과 값으로 본래 값을 구할 수도 있다.
# X_origin = std_scaler.inverse_transform(X_train_scaled)

In [53]:
X_train_scaled[0]

array([-0.45620424,  0.02795789, -0.31167784, -0.39923818, -0.18190779,
       -0.49906522, -0.36454482, -0.0193779 , -0.38960471, -0.47813089,
       -0.25956036, -0.56645338,  0.02414326,  0.04492438, -0.04812781,
       -0.03068375, -0.02749687, -0.02673015, -0.00239409,  0.18532114,
        0.07353428, -0.04976575, -0.10686775, -0.05710098, -0.02018284,
       -0.00960171, -0.0031203 , -0.25181912, -1.22993029,  0.98354463,
       -0.77563049, -0.03417307,  0.2758988 , -0.24321982, -0.14717149,
        0.08485812, -0.20841773, -0.10348383, -0.23451495, -0.57820507,
       -0.49383409, -0.62062374, -0.19067947, -0.09230915, -0.06799548])

# Validation도 Scale을 맞춰줘야겠지??

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


# fix random seed for reproducibility
numpy.random.seed(7)
# # load the dataset but only keep the top n words, zero the rest
# top_words = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# # truncate and pad input sequences
# max_review_length = 500
# X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# # create the model


embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))